In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import requests
import csv
import time

In [2]:
# ChromeDriver 옵션 설정
def initialize_webdriver():
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 제어 메시지 숨기기
    return webdriver.Chrome(options=options)

#### 이미지 url -> png 저장 함수

In [3]:
# 이미지 다운로드 및 저장 함수
def save_image_from_url(image_url, store_name, save_folder="images"):
    try:
        if image_url.startswith("//"):
            image_url = "https:" + image_url
        
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)
            
            image_name = f"{store_name}_img.png"  # 이미지 파일명: 가게명_img.png
            save_path = os.path.join(save_folder, image_name)

            with open(save_path, "wb") as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print(f"이미지가 {save_path}에 저장되었습니다.")
            return image_name  # 파일명 반환
        else:
            print(f"이미지 다운로드 실패: 상태 코드 {response.status_code}")
            return None
    except Exception as e:
        print(f"이미지 저장 중 오류 발생: {e}")
        return None


#### CSV 파일 저장 함수

In [4]:
# CSV 파일 저장 함수
def save_info_to_csv(info, menu_list, store_name):
    try:
        # 저장할 파일명
        info_filename = f"{store_name}_info.csv"
        menu_filename = f"{store_name}_menu.csv"

        # 정보 CSV 저장
        info_fieldnames = ["name", "category", "address", "hours", "review_count", "rating", "like_points", "image_filename"]
        with open(info_filename, mode="w", encoding="utf-8", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=info_fieldnames)
            writer.writeheader()
            writer.writerow(info)

        # 메뉴 CSV 저장
        menu_fieldnames = ["menu_name", "menu_price", "menu_description"]
        with open(menu_filename, mode="w", encoding="utf-8", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=menu_fieldnames)
            writer.writeheader()
            for menu in menu_list:
                writer.writerow(menu)

        print(f"정보 파일 '{info_filename}'와 메뉴 파일 '{menu_filename}'이 저장되었습니다.")
    except Exception as e:
        print(f"CSV 저장 중 오류 발생: {e}")

#### 상세 페이지 정보 크롤링 함수 

In [5]:
# 상세페이지에서 정보 크롤링 함수
def crawl_kakao_map_detail(driver):
    try:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        # 가게명
        name = soup.select_one("h2.tit_location").text.strip() if soup.select_one("h2.tit_location") else "이름 없음"

        # 대표 이미지 URL 추출
        image_url = ""
        try:
            image_element = soup.select_one("span.bg_present")
            if image_element:
                image_url = image_element["style"].split("url(")[-1].strip("')")
        except Exception as e:
            print(f"대표 이미지 URL 파싱 중 오류 발생: {e}")

        # 이미지 파일명 저장
        image_filename = save_image_from_url(image_url, name) if image_url else "대표이미지 없음"

        # 카테고리, 주소 등 크롤링
        category = soup.select_one("span.txt_location").text.strip() if soup.select_one("span.txt_location") else "카테고리 없음"
        address = soup.select_one("span.txt_address").text.strip() if soup.select_one("span.txt_address") else "주소 없음"

        # 영업시간
        hours = []
        hour_elements = soup.select("ul.list_operation li")
        for elem in hour_elements:
            hours.append(elem.text.strip())

        # 리뷰 수, 평점
        review_count = soup.select_one("a.link_evaluation[data-target='review']").get("data-cnt") if soup.select_one("a.link_evaluation[data-target='review']") else "리뷰 없음"
        rating = soup.select_one("span.color_b").text.strip() if soup.select_one("span.color_b") else "평점 없음"

        # 메뉴 정보
        menu_list = []
        menu_elements = soup.select("ul.list_menu li")
        for menu in menu_elements:
            menu_name = menu.select_one("span.loss_word").text.strip() if menu.select_one("span.loss_word") else "메뉴 이름 없음"
            price = menu.select_one("em.price_menu").text.strip() if menu.select_one("em.price_menu") else "가격 정보 없음"
            description = menu.select_one("p.txt_menu").text.strip() if menu.select_one("p.txt_menu") else "설명 없음"
            menu_list.append({"menu_name": menu_name, "menu_price": price, "menu_description": description})

        # 좋아요 포인트
        like_points = []
        point_elements = soup.select("span.chip_likepoint")
        for element in point_elements:
            category = element.select_one("span.txt_likepoint").text.strip() if element.select_one("span.txt_likepoint") else "항목 없음"
            score = element.select_one("span.num_likepoint").text.strip() if element.select_one("span.num_likepoint") else "0"
            like_points.append({"category": category, "score": score})

        # 결과 반환
        return {
            "name": name,
            "category": category,
            "address": address,
            "hours": hours,
            "review_count": review_count,
            "rating": rating,
            "like_points": like_points,
            "image_filename": image_filename
        }, menu_list

    except Exception as e:
        print(f"상세 정보 크롤링 중 오류 발생: {e}")
        return None, None

In [6]:
# 메인 실행
if __name__ == "__main__":
    driver = initialize_webdriver()
    try:
        # 1. 카카오맵 접속
        print("카카오맵 접속 중...")
        driver.get("https://map.kakao.com/")
        time.sleep(2)
        print("카카오맵 접속 완료!")

        # 2. 검색어 입력
        place_name = "쿳사 연희"  # 검색할 가게명
        print("검색창에 가게명 입력 중...")
        search_box = driver.find_element(By.ID, "search.keyword.query")
        search_box.send_keys(place_name)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)
        print(f"'{place_name}' 검색 완료!")

        # 3. 투명 레이어 제거
        try:
            dimmed_layer = driver.find_element(By.ID, "dimmedLayer")
            driver.execute_script("arguments[0].style.display = 'none';", dimmed_layer)
            print("투명 레이어 제거 완료!")
        except Exception as e:
            print("투명 레이어가 존재하지 않습니다. 계속 진행합니다.")

        # 4. 검색 결과에서 상세 페이지 링크 클릭
        print("검색 결과에서 상세 페이지 링크 클릭 대기 중...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.MediumTooltip a"))
        )
        print("상세 페이지 링크 찾기 완료!")

        print("상세 페이지 링크 클릭 중...")
        detail_link = driver.find_element(By.CSS_SELECTOR, "div.MediumTooltip a")
        detail_url = detail_link.get_attribute("href")  # 링크 주소 추출
        driver.get(detail_url)  # 상세페이지로 이동
        time.sleep(2)
        print("상세 페이지로 이동 완료!")

        # 상세페이지에서 정보 크롤링
        result, menu_list = crawl_kakao_map_detail(driver)
        if result:
            print("크롤링 결과 저장 중...")
            save_info_to_csv(result, menu_list, store_name=result["name"])

    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        driver.quit()
        print("브라우저 종료 완료.")

카카오맵 접속 중...
카카오맵 접속 완료!
검색창에 가게명 입력 중...
'쿳사 연희' 검색 완료!
투명 레이어 제거 완료!
검색 결과에서 상세 페이지 링크 클릭 대기 중...
상세 페이지 링크 찾기 완료!
상세 페이지 링크 클릭 중...
상세 페이지로 이동 완료!
이미지가 images/쿳사 연희_img.png에 저장되었습니다.
크롤링 결과 저장 중...
정보 파일 '쿳사 연희_info.csv'와 메뉴 파일 '쿳사 연희_menu.csv'이 저장되었습니다.
브라우저 종료 완료.
